## Introduction to Route Computation and Analysis using Batfish

Network engineers routinely need to validate routing and forwarding in the network. They often do that by connecting to multiple network devices and executing a series of `show route` commands. This distributed debugging is highly complex even in a moderately-sized network. Batfish makes this task extremely simple by providing an easy-to-query, centralized view of routing tables in the network. 

In this notebook, we will look at how you can extract routing information from Batfish.

Check out a video demo of this notebook [here](https://www.youtube.com/watch?v=AutkFa0xUxg).

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/intro-route-analysis?pixel&useReferer)

In [1]:
# Import packages and load questions
%run startup.py

 ### Initializing the Network and Snapshot

`SNAPSHOT_PATH` below can be updated to point to a custom snapshot directory, see the [Batfish instructions](https://github.com/batfish/batfish/wiki/Packaging-snapshots-for-analysis) for how to package data for analysis.<br>
More example networks are available in the [networks](https://github.com/batfish/batfish/tree/master/networks) folder of the Batfish repository.

In [2]:
# Initialize a network and snapshot
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "networks/example"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_snapshot'

The network snapshot that we initialized above is illustrated below. You can download/view devices' configuration files [here](https://github.com/batfish/pybatfish/tree/master/jupyter_notebooks/networks/example).

![example-network](https://raw.githubusercontent.com/batfish/pybatfish/master/jupyter_notebooks/networks/example/example-network.png)

All of the information we will show you in this notebook is dynamically computed by Batfish based on the configuration files for the network devices.

### View Routing Tables for ALL devices and ALL VRFs
Batfish makes **all** routing tables in the network easily accessible. Let's take a look at how you can retrieve the specific information you want.

In [3]:
# Get routing tables for all nodes and VRFs
routes_all = bfq.routes().answer().frame()

We are not going to print this table as it has a large number of entries. 

### View Routing Tables for default VRF on AS1 border routers

There are 2 ways that we can get the desired subset of data: 

Option 1) Only request that information from Batfish by passing in parameters into the `routes()` question. This is useful to do when you need to reduce the amount of data being returned, but is limited to regex filtering based on VRF, Node, Protocol and Network.

Option 2) Filter the output of the `routes()` question using the Pandas [APIs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.filter.html).

In [4]:
?bfq.routes

In [5]:
# Get the routing table for the 'default' VRF on border routers of as1
# using BF parameters
routes_as1border = bfq.routes(nodes="as1border.*", vrfs="default").answer().frame()

In [6]:
# Get the routing table for the 'default' VRF on border routers of as1
# using Pandas filtering
routes_as1border = routes_all[(routes_all['Node'].str.contains('as1border')) & (routes_all['VRF'] == 'default')]
routes_as1border

Node      VRF        Network   Protocol     Next_Hop_IP  \
1    as1border1  default  3.0.1.0/24     ibgp       10.13.22.3       
6    as1border1  default  10.12.11.0/24  connected  AUTO/NONE(-1l)   
7    as1border2  default  10.13.22.0/24  connected  AUTO/NONE(-1l)   
11   as1border2  default  1.2.2.2/32     connected  AUTO/NONE(-1l)   
46   as1border1  default  1.0.1.0/24     connected  AUTO/NONE(-1l)   
54   as1border2  default  3.0.1.0/24     bgp        10.13.22.3       
58   as1border1  default  1.0.1.1/32     local      AUTO/NONE(-1l)   
125  as1border2  default  1.1.1.1/32     ospf       1.0.2.2          
128  as1border1  default  1.1.1.1/32     connected  AUTO/NONE(-1l)   
161  as1border2  default  10.14.22.1/32  local      AUTO/NONE(-1l)   
185  as1border1  default  10.13.22.0/24  ospfE2     1.0.1.2          
214  as1border1  default  3.0.2.0/24     ibgp       10.13.22.3       
219  as1border1  default  1.2.2.2/32     ospf       1.0.1.2          
220  as1border2  default  1.0.1.0/24     ospf       1.0.2.2          
221  as1border1  default  1.10.1.1/32    ospf       1.0.1.2          
225  as1border2  default  3.0.2.0/24     bgp        10.13.22.3       
226  as1border2  default  10.14.22.0/24  connected  AUTO/NONE(-1l)   
227  as1border1  default  2.128.0.0/16   bgp        10.12.11.2       
228  as1border2  default  10.12.11.0/24  ospfE2     1.0.2.2          
248  as1border2  default  1.10.1.1/32    ospf       1.0.2.2          
251  as1border2  default  2.128.0.0/16   ibgp       10.12.11.2       
280  as1border1  default  1.0.2.0/24     ospf       1.0.1.2          
286  as1border2  default  1.0.2.1/32     local      AUTO/NONE(-1l)   
290  as1border2  default  10.13.22.1/32  local      AUTO/NONE(-1l)   
300  as1border1  default  10.14.22.0/24  ospfE2     1.0.1.2          
320  as1border2  default  1.0.2.0/24     connected  AUTO/NONE(-1l)   
336  as1border1  default  10.12.11.1/32  local      AUTO/NONE(-1l)   

       Next_Hop  Admin_Distance  Metric   Tag  
1    as3border2  200             50      None  
6    None        0               0       None  
7    None        0               0       None  
11   None        0               0       None  
46   None        0               0       None  
54   as3border2  20              50      None  
58   None        0               0       None  
125  as1core1    110             3       None  
128  None        0               0       None  
161  None        0               0       None  
185  as1core1    110             20      None  
214  as3border2  200             50      None  
219  as1core1    110             3       None  
220  as1core1    110             2       None  
221  as1core1    110             2       None  
225  as3border2  20              50      None  
226  None        0               0       None  
227  as2border1  20              50      None  
228  as1core1    110             20      None  
248  as1core1    110             2       None  
251  as2border1  200             50      None  
280  as1core1    110             2       None  
286  None        0               0       None  
290  None        0               0       None  
300  as1core1    110             20      None  
320  None        0               0       None  
336  None        0               0       None

### View BGP learnt routes for default VRF on AS1 border routers

In [7]:
# Getting BGP routes in the routing table for the 'default' VRF on border routers of as1
# using BF parameters
routes_as1border_bgp = bfq.routes(nodes="as1border.*", vrfs="default", protocols="bgp").answer().frame()

In [8]:
# Geting BGP routes in the routing table for the 'default' VRF on border routers of as1
# using Pandas filtering
routes_as1border_bgp = routes_all[(routes_all['Node'].str.contains('as1border')) & (routes_all['VRF'] == 'default') & (routes_all['Protocol'] == 'bgp')]
routes_as1border_bgp

Node      VRF       Network Protocol Next_Hop_IP    Next_Hop  \
54   as1border2  default  3.0.1.0/24    bgp      10.13.22.3  as3border2   
225  as1border2  default  3.0.2.0/24    bgp      10.13.22.3  as3border2   
227  as1border1  default  2.128.0.0/16  bgp      10.12.11.2  as2border1   

     Admin_Distance  Metric   Tag  
54   20              50      None  
225  20              50      None  
227  20              50      None

### View BGP learnt routes for ALL VRFs on ALL routers with Metric >=50
We cannot pass in `metric` as a parameter to Batfish, so this task is best handled with the Pandas API.

In [9]:
routes_filtered = routes_all[(routes_all['Protocol'] == 'bgp') & (routes_all['Metric'] >= 50)]
routes_filtered

Node      VRF       Network Protocol Next_Hop_IP    Next_Hop  \
12   as2dist2    default  2.128.0.0/24  bgp      2.34.201.4  as2dept1     
24   as2dept1    default  1.0.1.0/24    bgp      2.34.201.3  as2dist2     
54   as1border2  default  3.0.1.0/24    bgp      10.13.22.3  as3border2   
59   as2dept1    default  3.0.1.0/24    bgp      2.34.201.3  as2dist2     
62   as3border1  default  2.128.0.0/16  bgp      10.23.21.2  as2border2   
77   as2dist1    default  2.128.1.0/24  bgp      2.34.101.4  as2dept1     
105  as2dist1    default  2.128.0.0/24  bgp      2.34.101.4  as2dept1     
124  as2border2  default  3.0.1.0/24    bgp      10.23.21.3  as3border1   
130  as2dept1    default  1.0.1.0/24    bgp      2.34.101.3  as2dist1     
138  as2border2  default  3.0.2.0/24    bgp      10.23.21.3  as3border1   
195  as2dept1    default  1.0.2.0/24    bgp      2.34.201.3  as2dist2     
204  as2dept1    default  3.0.2.0/24    bgp      2.34.201.3  as2dist2     
225  as1border2  default  3.0.2.0/24    bgp      10.13.22.3  as3border2   
227  as1border1  default  2.128.0.0/16  bgp      10.12.11.2  as2border1   
233  as2dept1    default  1.0.2.0/24    bgp      2.34.101.3  as2dist1     
268  as3border2  default  1.0.2.0/24    bgp      10.13.22.1  as1border2   
276  as2border1  default  1.0.2.0/24    bgp      10.12.11.1  as1border1   
284  as2dept1    default  3.0.2.0/24    bgp      2.34.101.3  as2dist1     
292  as2border1  default  1.0.1.0/24    bgp      10.12.11.1  as1border1   
316  as2dept1    default  3.0.1.0/24    bgp      2.34.101.3  as2dist1     
323  as2dist2    default  2.128.1.0/24  bgp      2.34.201.4  as2dept1     
328  as3border2  default  1.0.1.0/24    bgp      10.13.22.1  as1border2   

     Admin_Distance  Metric   Tag  
12   20              50      None  
24   20              50      None  
54   20              50      None  
59   20              50      None  
62   20              50      None  
77   20              50      None  
105  20              50      None  
124  20              50      None  
130  20              50      None  
138  20              50      None  
195  20              50      None  
204  20              50      None  
225  20              50      None  
227  20              50      None  
233  20              50      None  
268  20              50      None  
276  20              50      None  
284  20              50      None  
292  20              50      None  
316  20              50      None  
323  20              50      None  
328  20              50      None

### View the routing entries for network 1.0.2.0/24 on ALL routers in ALL VRFs

In [10]:
# grab the route table entry for network 1.0.2.0/24 from all routers in all VRFs
# using BF parameters
routes_filtered = bfq.routes(network="1.0.2.0/24").answer().frame()

In [11]:
# grab the route table entry for network 1.0.2.0/24 from all routers in all VRFs
# using Pandas filtering
routes_filtered = routes_all[routes_all['Network'] == "1.0.2.0/24"]
routes_filtered

Node      VRF     Network   Protocol     Next_Hop_IP    Next_Hop  \
127  as2core2    default  1.0.2.0/24  ibgp       10.12.11.1      as1border1   
133  as2core1    default  1.0.2.0/24  ibgp       10.12.11.1      as1border1   
195  as2dept1    default  1.0.2.0/24  bgp        2.34.201.3      as2dist2     
233  as2dept1    default  1.0.2.0/24  bgp        2.34.101.3      as2dist1     
244  as2dist1    default  1.0.2.0/24  ibgp       10.12.11.1      as1border1   
245  as2dist1    default  1.0.2.0/24  ibgp       10.12.11.1      as1border1   
246  as2dist2    default  1.0.2.0/24  ibgp       10.12.11.1      as1border1   
247  as2dist2    default  1.0.2.0/24  ibgp       10.12.11.1      as1border1   
253  as3border1  default  1.0.2.0/24  ibgp       10.13.22.1      as1border2   
268  as3border2  default  1.0.2.0/24  bgp        10.13.22.1      as1border2   
271  as2border2  default  1.0.2.0/24  ibgp       10.12.11.1      as1border1   
272  as2border2  default  1.0.2.0/24  ibgp       10.12.11.1      as1border1   
276  as2border1  default  1.0.2.0/24  bgp        10.12.11.1      as1border1   
280  as1border1  default  1.0.2.0/24  ospf       1.0.1.2         as1core1     
293  as1core1    default  1.0.2.0/24  connected  AUTO/NONE(-1l)  None         
294  as3core1    default  1.0.2.0/24  ibgp       10.13.22.1      as1border2   
320  as1border2  default  1.0.2.0/24  connected  AUTO/NONE(-1l)  None         

     Admin_Distance  Metric   Tag  
127  200             50      None  
133  200             50      None  
195  20              50      None  
233  20              50      None  
244  200             50      None  
245  200             50      None  
246  200             50      None  
247  200             50      None  
253  200             50      None  
268  20              50      None  
271  200             50      None  
272  200             50      None  
276  20              50      None  
280  110             2       None  
293  0               0       None  
294  200             50      None  
320  0               0       None

Using Panda's filtering it is easy to retrieve the list of nodes which have the network in the routing table for at least 1 VRF. This type of processing should always be done using the Pandas APIs.

In [12]:
# Get the list of nodes that have the network 1.0.2.0/24 in at least 1 VRF
# the .unique function removes duplicate entries that would have been returned if the network was in multiple VRFs on a node or there were
# multiple route entries for the network (ECMP)
print(sorted(routes_filtered["Node"].unique()))

['as1border1', 'as1border2', 'as1core1', 'as2border1', 'as2border2', 'as2core1', 'as2core2', 'as2dept1', 'as2dist1', 'as2dist2', 'as3border1', 'as3border2', 'as3core1']


Now we will retrieve the list of nodes that do NOT have this prefix in their routing table. This is easy to do with the Pandas [groupby](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) and [filter](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.filter.html) functions.

In [13]:
# Group all routes by Node and filter for those that don't have '1.0.2.0/24'
routes_filtered = routes_all.groupby('Node').filter(lambda x: all(x['Network'] != '1.0.2.0/24'))

# Get the unique node names and sort the list
print(sorted(routes_filtered["Node"].unique()))

['host1', 'host2']


The only devices that do not have a route to **1.0.2.0/24** are the 2 hosts in the snapshot. This is expected, as they should just have a default route. Let's verify that.

In [14]:
routes_all[routes_all['Node'].str.contains('host')]

Node      VRF         Network   Protocol     Next_Hop_IP  Next_Hop  \
0    host2  default  2.128.1.0/24    connected  AUTO/NONE(-1l)  None       
63   host1  default  2.128.0.101/32  local      AUTO/NONE(-1l)  None       
107  host2  default  2.128.1.101/32  local      AUTO/NONE(-1l)  None       
200  host2  default  0.0.0.0/0       static     2.128.1.1       as2dept1   
309  host1  default  2.128.0.0/24    connected  AUTO/NONE(-1l)  None       
317  host1  default  0.0.0.0/0       static     2.128.0.1       as2dept1   

     Admin_Distance  Metric   Tag  
0    0               0       None  
63   0               0       None  
107  0               0       None  
200  1               0       None  
309  0               0       None  
317  1               0       None

With Batfish and Pandas you can easily retrieve the exact information you are looking for from the routing tables on ANY or ALL network devices for ANY or ALL VRFs.

This concludes the notebook. To recap, in this notebook we covered the foundational tasks for route analysis:

1. How to get routes at all nodes in the network or only at a subset of them
2. How to retrieve routing entries that match a specific protocol or metric
3. How to find which nodes have an entry for a prefix or which ones do not


We hope you found this notebook useful and informative. Future notebooks will dive into more advanced topics like path analysis, debugging ACLs and firewall rules, validating routing policy, etc.. so stay tuned! 

### Want to know more? 

Reach out to us through [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) or [Github](https://github.com/batfish/batfish) to learn more, or send feedback.